In [1]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
%pylab inline
import warnings
warnings.filterwarnings('ignore')
import random

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('Train_rev1.csv')

In [3]:
random.seed(99) 
sample = random.sample(range(len(data)),2500)

In [4]:
data_s = data.iloc[sample]

### B1
#### Model with numeric columns only

In [5]:
col_names = ['location','contract_type','contract_time','type_of_job','salary']

In [6]:
num_data = data_s[['LocationNormalized','ContractType','ContractTime','Category','SalaryNormalized']]
num_data.columns = col_names

In [7]:
num_data.head()

,location,contract_type,contract_time,type_of_job,salary
98880,Billericay,NaN,contract,Travel Jobs,17568
48972,Crawley,NaN,NaN,Admin Jobs,23500
43765,London,NaN,NaN,Hospitality & Catering Jobs,30000
60808,Warrington,NaN,permanent,HR & Recruitment Jobs,31000
185993,London,NaN,permanent,Legal Jobs,31500


Create Target

In [8]:
p=np.percentile(num_data['salary'],75)
def target(t):
    if t>p:
        return 1
    else:
        return 0
    
num_data['target'] = num_data['salary'].map(target)

Get a list of cities with highest cost of living.

In [9]:
#Get the top 10 highest CoL data from https://abcfinance.co.uk/blog/the-true-cost-of-living-in-uk-cities/
high_cost = ['London','Milton Keynes','Bath','Reading','Aberdeen','Cambridge','Oxford','Portsmouth','Edinburgh','York']

In [10]:
def location_class(s):
    if s in high_cost:
        return 'high'
    else:
        return 'low'
num_data['location_class'] = num_data['location'].map(location_class)

Now, get the dummies for each variables.

In [11]:
print num_data.columns.values

['location' 'contract_type' 'contract_time' 'type_of_job' 'salary'
 'target' 'location_class']


In [13]:
categorical_columns = ['contract_type', 'contract_time', 'type_of_job','location_class']
data_dummies = pd.get_dummies(num_data[categorical_columns],
                            prefix=categorical_columns,
                            columns=categorical_columns)
dummy_column_names = data_dummies.columns.values
dummy_column_names[:10]

array(['contract_type_full_time', 'contract_type_part_time',
       'contract_time_contract', 'contract_time_permanent',
       'type_of_job_Accounting & Finance Jobs', 'type_of_job_Admin Jobs',
       'type_of_job_Charity & Voluntary Jobs',
       'type_of_job_Consultancy Jobs',
       'type_of_job_Creative & Design Jobs',
       'type_of_job_Customer Services Jobs'], dtype=object)

In [14]:
num_data2 = pd.concat([num_data, data_dummies], axis=1)
num_data2.columns.values

array(['location', 'contract_type', 'contract_time', 'type_of_job',
       'salary', 'target', 'location_class', 'contract_type_full_time',
       'contract_type_part_time', 'contract_time_contract',
       'contract_time_permanent', 'type_of_job_Accounting & Finance Jobs',
       'type_of_job_Admin Jobs', 'type_of_job_Charity & Voluntary Jobs',
       'type_of_job_Consultancy Jobs',
       'type_of_job_Creative & Design Jobs',
       'type_of_job_Customer Services Jobs',
       'type_of_job_Domestic help & Cleaning Jobs',
       'type_of_job_Energy, Oil & Gas Jobs',
       'type_of_job_Engineering Jobs', 'type_of_job_Graduate Jobs',
       'type_of_job_HR & Recruitment Jobs',
       'type_of_job_Healthcare & Nursing Jobs',
       'type_of_job_Hospitality & Catering Jobs', 'type_of_job_IT Jobs',
       'type_of_job_Legal Jobs', 'type_of_job_Logistics & Warehouse Jobs',
       'type_of_job_Maintenance Jobs', 'type_of_job_Manufacturing Jobs',
       'type_of_job_Other/General Jobs',
    

Now, build Naive Bayes model.

In [16]:
formula = 'target ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in dummy_column_names]))

In [17]:
Y, X = dmatrices(formula, num_data2, return_type='dataframe')
y = Y['target'].values

In [18]:
from sklearn import naive_bayes
model = naive_bayes.MultinomialNB()
model.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
from sklearn import metrics

prediction_train = model.predict(X)
print metrics.accuracy_score(y, prediction_train)

0.7524


The accuracy of using numerical variables are 75.24%

In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, prediction_train)

array([[1802,   73],
       [ 546,   79]])

#### Model with text columns only